In [1]:
import os
import json
from selenium import webdriver
from datetime import datetime, timedelta
from booking import getBookingDataForCheckingInOut
from expedia import getExpediaDataForCheckingInOut
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

chromeDriverPath = "./driver/chromedriver"
chromeOptions = Options()
for arg in ["--no-sandbox", "--disable-dev-shm-usage"]:
    chromeOptions.add_argument(arg)

In [2]:
def collectDataForRange(range, isBooking):
    dataList = []
    startDate, endDate = range

    while startDate < endDate:
        checkIn = startDate.strftime("%Y-%m-%d")
        checkOut = (startDate + timedelta(days=1)).strftime("%Y-%m-%d")
        service = Service(chromeDriverPath)
        driver = webdriver.Chrome(service=service, options=chromeOptions)
        try:

            data = (
                getBookingDataForCheckingInOut(driver, checkIn, checkOut)
                if isBooking
                else getExpediaDataForCheckingInOut(driver, checkIn, checkOut)
            )
            print(f"{len(data)} hotels available from {checkIn} to {checkOut}")
        except Exception:
            data = []
        driver.quit()
        dataList.extend(data)
        startDate += timedelta(days=1)
    return dataList

In [3]:
start, end = datetime(2024, 10, 1), datetime(2024, 10, 31)


def getDataList(start, end, isBooking):
    pholder = "booking" if isBooking else "expedia"
    dataFile = f"{pholder}_{start.strftime('%Y-%m-%d')}_{end.strftime('%Y-%m-%d')}.json"
    if os.path.exists(dataFile):
        with open(dataFile, "r") as jsonFile:
            return json.load(jsonFile)
    else:
        comData = collectDataForRange((start, end), isBooking)
        with open(dataFile, "w") as jsonFile:
            json.dump(comData, jsonFile, indent=4)
        return comData


bookingData = getDataList(start, end, isBooking=True)
expediaData = getDataList(start, end, isBooking=False)